In [2]:
#starting with simple assumptions
'''For our example we will 
use text from Yelp reviews to 
predict the star rating given 
by the user'''

import pandas as pd

df = pd.read_csv("assets/data/yelp.csv.gz")
df.shape

(10000, 10)

In [3]:
df.head(5)

,business_id,date,review_id,stars,text,type,user_id,cool,useful,funny
0,9yKzy9PApeiPPOUJEtnvkg,2011-01-26,fWKvX83p0-ka4JS3dc6E5A,5,My wife took me here on my birthday for breakf...,review,rLtl8ZkDX5vH5nAx9C3q5Q,2,5,0
1,ZRJwVLyzEJq1VAihDhYiow,2011-07-27,IjZ33sJrzXqU-0X6U8NwyA,5,I have no idea why some people give bad review...,review,0a2KyEL0d3Yb1V6aivbIuQ,0,0,0
2,6oRAC4uyJCsJl1X0WZpVSA,2012-06-14,IESLBzqUCLdSzSqm0eCSxQ,4,love the gyro plate. Rice is so good and I als...,review,0hT2KtfLiobPvh6cDC8JQg,0,1,0
3,_1QQZuf4zZOyFCvXc0o6Vg,2010-05-27,G-WvGaISbqqaMHlNnByodA,5,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",review,uZetl9T0NcROGOyFfughhg,1,2,0
4,6ozycU1RpktNG2-1BroVtw,2012-01-05,1uJFq2r5QfJG_6ExMRCaGw,5,General Manager Scott Petello is a good egg!!!...,review,vYmM4KTsC8ZfQBg-j5MWkw,0,0,0


In [4]:
reviews = df["text"].values
star_ratings = df["stars"].values
print(reviews[0])
print(star_ratings[0])

My wife took me here on my birthday for breakfast and it was excellent.  The weather was perfect which made sitting outside overlooking their grounds an absolute pleasure.  Our waitress was excellent and our food arrived quickly on the semi-busy Saturday morning.  It looked like the place fills up pretty quickly so the earlier you get here the better.

Do yourself a favor and get their Bloody Mary.  It was phenomenal and simply the best I've ever had.  I'm pretty sure they only use ingredients from their garden and blend them fresh when you order it.  It was amazing.

While EVERYTHING on the menu looks excellent, I had the white truffle scrambled eggs vegetable skillet and it was tasty and delicious.  It came with 2 pieces of their griddled bread with was amazing and it absolutely made the meal complete.  It was the best "toast" I've ever had.

Anyway, I can't wait to go back!
5


In [5]:
words = reviews[0].split()
words[:10]

['My',
 'wife',
 'took',
 'me',
 'here',
 'on',
 'my',
 'birthday',
 'for',
 'breakfast']

In [6]:
lower_words = [w.lower() for w in words]
lower_words[0:10]

['my',
 'wife',
 'took',
 'me',
 'here',
 'on',
 'my',
 'birthday',
 'for',
 'breakfast']

In [7]:
from nltk.corpus import stopwords as nltk_stopwords

stopwords = nltk_stopwords.words('english')
print(len(stopwords))
stopwords[:10]

179


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

In [8]:
print("number of words in the original document", len(set(lower_words)))
useful_words = [word for word in lower_words if word not in stopwords]
print("number of words in the original document, excluding stopwords", len(set(useful_words)))

number of words in the original document 106
number of words in the original document, excluding stopwords 76


In [9]:
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer('english')
stemmer.stem("having")

'have'

In [10]:
stemmed_words = [stemmer.stem(word) for word in useful_words]
print(useful_words[:10])
print(stemmed_words[:10])

['wife', 'took', 'birthday', 'breakfast', 'excellent.', 'weather', 'perfect', 'made', 'sitting', 'outside']
['wife', 'took', 'birthday', 'breakfast', 'excellent.', 'weather', 'perfect', 'made', 'sit', 'outsid']


In [11]:
# apply stemming
df["text_stemmed"] = df["text"].apply(lambda x: " ".join([stemmer.stem(w) for w in x.split()]))
print(df["text"].values[0])
print(df["text_stemmed"].values[0])

My wife took me here on my birthday for breakfast and it was excellent.  The weather was perfect which made sitting outside overlooking their grounds an absolute pleasure.  Our waitress was excellent and our food arrived quickly on the semi-busy Saturday morning.  It looked like the place fills up pretty quickly so the earlier you get here the better.

Do yourself a favor and get their Bloody Mary.  It was phenomenal and simply the best I've ever had.  I'm pretty sure they only use ingredients from their garden and blend them fresh when you order it.  It was amazing.

While EVERYTHING on the menu looks excellent, I had the white truffle scrambled eggs vegetable skillet and it was tasty and delicious.  It came with 2 pieces of their griddled bread with was amazing and it absolutely made the meal complete.  It was the best "toast" I've ever had.

Anyway, I can't wait to go back!
my wife took me here on my birthday for breakfast and it was excellent. the weather was perfect which made sit

In [12]:
from sklearn.model_selection import train_test_split

X = df.loc[df["stars"].isin([1, 5]), "text_stemmed"]
y = df.loc[df["stars"].isin([1, 5]), "stars"]

# stratify keeps the proportions
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
y.value_counts()

5    3337
1     749
Name: stars, dtype: int64

In [13]:
# in order to use LogisticRegression we must have numerical values as X_train
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(X_train, y_train)

ValueError: could not convert string to float: "yikes, read other review i realiz my bad experi wasn't unique. as a server i make a veri laid back customer. i like pretti much everyth i eat and don't requir a lot of attent from the waiter. la piccola cucina would benefit from just one extra person in the front of the house. our guy, though ador and friendly, was too busi to refil our drink and to rememb to bring our appet (though charg us for it). the ahi tuna (high recommend over the other fish option he said) was so overcook it was the color and consist of chicken. like other review mentioned, he was frantic and made that clear to everi customer. at one point i even saw him in the kitchen cook - they need anoth person! i left super stress out from the experience, which is very, veri unusu for me. you can either have bad servic or bad food, but not both."

In [14]:
from sklearn.feature_extraction.text import CountVectorizer

vec = CountVectorizer(binary=True,
                      stop_words='english',
                      lowercase=True # default
                     )

# starting from our 2860 documents we took for training set, we translate them into bag of words, 
# i.e. dictionaries of word count
X_train_text = vec.fit_transform(X_train)
X_test_text = vec.transform(X_test)

print(len(vec.vocabulary_))
# look at some random features
print(vec.get_feature_names()[1000:1010])

14379
['astound', 'astounded', 'astounding', 'astrological', 'astronom', 'asu', 'atari', 'ate', 'atf', 'athelet']


In [15]:
# in order to use LogisticRegression we must have numerical values as X_train
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(X_train_text, y_train)
print(lr.intercept_, lr.coef_)

[1.13192726] [[ 2.39703753e-01  4.95926015e-02 -1.36257012e-03 ...  2.78036103e-03
   8.09948162e-06  6.41188584e-05]]


In [16]:
from sklearn.metrics import accuracy_score

y_pred = lr.predict(X_test_text)

accuracy_score(y_test, y_pred)

0.9290375203915171

In [ ]:
#Exersize